## Walk through of a sample PCL code:

### Lesson 2 Volume management

[volume.py](../../volume.py)

#### The Header Comment Section

This section usually includes details such as 
- the file name, 
- author, 
- date, 
- purpose of the program, and any other relevant information. 

It’s a good practice to include such comments for better readability and maintainability of the code 

In [ ]:
#! /usr/local/bin/python3

"""
ONTAP 9.13.1 REST API Python Client Library Scripts
uthor: Vish Hulikal
This script performs the following:
        - Create a volume
        - List all volumes
        - Resize a volume to a new (bigger) size
        - Delete a volume

usage: python3.11 volume.py [-h] -c CLUSTER -v VOLUME_NAME -vs VSERVER_NAME -a AGGR_NAME
               -rs VOLUME_RESIZE -s VOLUME_SIZE [-u API_USER] [-p API_PASS]
The following arguments are required: -c/--cluster, -v/--volume_name, -vs/--vserver_name,
                -a/--aggr_name, -rs/--volume_resize, -s/--volume_size
"""


#### The Input Section:

Since we are using a Jupyter Notebook to run the code and not running the code from the command line, we need this section to simulate entering the arguments from the command line.

When you run the notebook, for example by clicking on the `Run All` button, a dialog prompt will appear at the top of the window. You will then need to enter the command line arguments as described in the usage section above.

If you do not provide the user, `admin`is used. If you do not provide the password, you will be prompted for the password.

Suggested command arguments:

In [ ]:
import sys

# Prompt the user to enter command line arguments
args = input("Please enter command line arguments: ")

# Split the entered string into a list of arguments
args = args.split()

# Assign the list of arguments to sys.argv
sys.argv = ['ipykernel_launcher.py'] + args

#### Import Section

Here we will import the following modules:
  - [`argparse`](https://pypi.org/project/argparse/) : This is a popular python module. The argparse module makes it easy to write user friendly command line interfaces. 
  The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.
  - [`getpass`](https://docs.python.org/3/library/getpass.html) : Used to Prompt the user for a password without echoing
  - [`logging`](https://pypi.org/project/logging/) : This module is intended to provide a standard error logging mechanism in Python as per PEP 282.
  - [`netapp_ontap.config`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/config.html) : This module contains the global configuration options and related functions for the library.
  - [`netapp_ontap.host_connection`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/host_connection.html) : This module defines a host connection object which is used to communicate with the API host
  - [`netapp_ontap.error`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/error.html) : This module defines the custom exception type. All exceptions raised by the library descend from this type
  - [`netapp_ontap.resources.volume`](https://library.netapp.com/ecmdocs/ECMLP3319064/html/resources/volume.html) 


In [ ]:

import argparse
from getpass import getpass
import logging

from netapp_ontap import config, HostConnection, NetAppRestError
from netapp_ontap.resources import Volume


#### Function Definitions

There functions are defined:
  - `create_volume`  
  - `list_volumes`
  - `get_volume`
  - `delete_volume`
  - `resize_volume`

In [ ]:

def create_volume(volume_name: str, vserver_name: str, aggr_name: str, volume_size: int) -> None:
    """Creates a new volume in a SVM"""

    data = {
        'name': volume_name,
        'svm': {'name': vserver_name},
        'aggregates': [{'name': aggr_name }],
        'size': volume_size
    }

    volume = Volume(**data)

    try:
        volume.post()
        print("Volume %s created successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume was not created: %s" % err)
    return

def list_volumes(vserver_name: str) -> None:
    """List Volumes in a SVM """

    print ("\nList of Volumes:-")
    try:
        for volume in Volume.get_collection(**{"svm.name": vserver_name}):
            volume.get()
            print (volume.name)
    except NetAppRestError as err:
        print("Error: Volume list was not created: %s" % err)
    return

def get_volume(volume_name: str) -> None:
    """Get the details of a volume"""

    volume = Volume.find(name=volume_name)

    try:
        volume.get()
        print (volume.name)
        print (volume.uuid)
        
        print("Volume details for %s obtained successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume details not obtained: %s" % err)
    return

def delete_volume(vserver_name: str, volume_name: str) -> None:
    """Delete a volume in a SVM"""

    volume = Volume.find(name=volume_name, svm={"name":vserver_name})
    if (volume == None):
        print ("Error: Volume %s not found!" % volume.name)
        return

    try:
        volume.delete()
        print("Volume %s deleted successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume was not deleted: %s" % err)
    return

def resize_volume(vserver_name: str, volume_name: str, volume_resize: int) -> None:
    """Resize a volume in a SVM"""

    volume = Volume.find(name=volume_name, svm={"name":vserver_name})
    if (volume == None):
        print ("Error: Volume %s not found!" % volume.name)
        return

    volume.size = volume_resize

    try:
        volume.patch()
        print("Volume %s resized successfully" % volume.name)
    except NetAppRestError as err:
        print("Error: Volume was not resized: %s" % err)
    return


#### Arguments Parser

We define which arguments need to be passed to the script and argparse does the rest...

In [ ]:
def parse_args() -> argparse.Namespace:
    """Parse the command line arguments from the user"""

    parser = argparse.ArgumentParser(
        description="This script will create a new volume."
    )
    parser.add_argument(
        "-c", "--cluster", required=True, help="API server IP:port details"
    )
    parser.add_argument(
        "-v", "--volume_name", required=True, help="Volume to create or clone from"
    )
    parser.add_argument(
        "-vs", "--vserver_name", required=True, help="SVM to create the volume from"
    )
    parser.add_argument(
        "-a", "--aggr_name", required=True, help="Aggregate to create the volume from"
    )
    parser.add_argument(
        "-rs", "--volume_resize", required=True, help="Volume resize in bytes"
    )
    parser.add_argument(
        "-s", "--volume_size", required=True, help="Size of the volume."
    )
    parser.add_argument("-u", "--api_user", default="admin", help="API Username")
    parser.add_argument("-p", "--api_pass", help="API Password")
    parsed_args = parser.parse_args()

    # collect the password without echo if not already provided
    if not parsed_args.api_pass:
        parsed_args.api_pass = getpass()

    return parsed_args


#### The Main Section

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(
        level=logging.INFO,
        format="[%(asctime)s] [%(levelname)5s] [%(module)s:%(lineno)s] %(message)s",
    )
    args = parse_args()
    config.CONNECTION = HostConnection(
        args.cluster, username=args.api_user, password=args.api_pass, verify=False,
    )


In [ ]:

    # Create a Volume
    create_volume(args.volume_name, args.vserver_name, args.aggr_name , args.volume_size)


In [ ]:

    # List all volumes in the VServer
    list_volumes(args.vserver_name)


In [ ]:

    # Get the volume details
    get_volume(args.volume_name)


In [ ]:

    # Resize a volume
    resize_volume(args.vserver_name, args.volume_name, args.volume_resize)


In [ ]:

    # Delete a volume
    delete_volume(args.vserver_name, args.volume_name)



---

[Back to Module 2 Demo - Notebook:](./Module2_demo.ipynb)